In [ ]:
### Please use this when running code in Google Colab.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
### Module ###
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas(desc="progress: ")

In [ ]:
###
### Join SHAP-val and Test-val data for one CM label (Confusion Matrix: TP, FN, FP, TN) ###
###
def join_data_one_cmlabel(cm_label: str):

    bar = tqdm(total=90)
    bar.set_description("Progress")

    df_all_time_point = pd.DataFrame()

    for time_point in range(1, 90+1):

        file_shap  = "/content/drive/MyDrive/res_death_destiny/data/shap_values/shap_values_"+cm_label+"/shap_values_"+cm_label+"_tp_"+str(time_point)+".pkl"
        file_test = "/content/drive/MyDrive/res_death_destiny/data/test_values/test_values_"+cm_label+"/test_values_"+cm_label+"_tp_"+str(time_point)+".pkl"

        df_shap  = pd.read_pickle(file_shap)
        df_test = pd.read_pickle(file_test)

        columns = list(df_shap.columns)

        df_one_time_point = pd.DataFrame()

        for test_item in columns[:-1]:

            df_shap_one_item = df_shap.loc[:, [test_item, "Patient_ID"]]
            df_test_one_item = df_test.loc[:, [test_item, "Patient_ID"]]

            df_shap_one_item = df_shap_one_item.rename(columns={test_item:"SHAP_value"})
            df_test_one_item = df_test_one_item.rename(columns={test_item:"Test_value"})

            df_one_item = pd.merge(df_shap_one_item, df_test_one_item, on="Patient_ID")

            df_one_item["Test_item"] = test_item

            df_one_time_point = pd.concat([df_one_time_point, df_one_item])

        df_one_time_point["Time_point"] = time_point

        df_all_time_point = pd.concat([df_all_time_point, df_one_time_point])

        bar.update(1)

    df_all_time_point = df_all_time_point.reindex(columns=["Patient_ID","Test_item","Test_value","SHAP_value","Time_point"]).reset_index(drop=True)

    filename = "shap_test_values_"+cm_label+".pkl"
    df_all_time_point.to_pickle("/content/drive/MyDrive/res_death_destiny/data/shap_test_values/"+filename)


In [ ]:
###
### Join SHAP-val and Test-val data for one CM label (TP, FN, FP, TN) ###
###
for cm_label in ["tp", "fn", "fp", "tn"]:
  join_data_one_cmlabel(cm_label=cm_label)

In [ ]:
###
### Max Scaling (SHAP values) ###
###
def Scaling_shap_values():

    ls_max = []

    ### Get Max SHAP value for one CM label ###
    for cm_label in ["tp", "fn", "fp", "tn"]:

        filename = "shap_test_values_"+cm_label+".pkl"
        df = pd.read_pickle("/content/drive/MyDrive/res_death_destiny/data/shap_test_values/"+filename)
        ls_max.append(df.loc[:,"SHAP_value"].max())

    ### Calculate Max SHAP value ###
    max = sorted(ls_max, reverse=True)[0]

    ### Max Scaling ###
    for cm_label in ["tp", "fn", "fp", "tn"]:

        filename = "shap_test_values_"+cm_label+".pkl"
        df = pd.read_pickle("/content/drive/MyDrive/res_death_destiny/data/shap_test_values/"+filename)
        df_shap_drop = df.drop(["SHAP_value"], axis=1)
        df_shap = df.loc[:,"SHAP_value"]
        df_shap_norm = df_shap / max
        df_shap_norm = df_shap_norm.rename("SHAP_value")
        df_norm = pd.concat([df_shap_drop, df_shap_norm], axis="columns")
        df_norm = df_norm.reindex(columns=["Patient_ID","Test_item","Test_value","SHAP_value","Time_point"])
        df_norm.to_pickle("/content/drive/MyDrive/res_death_destiny/data/shap_test_values/shap_test_values_norm_"+cm_label+".pkl")



In [ ]:
###
### Max Scaling (SHAP values) ###
###
Scaling_shap_values()